In [12]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import psycopg2
import decimal
import pandas as pd

In [2]:
Base = automap_base()
connection_string = "gtuser:rpython2020@104.197.208.53:5432/covid"
engine = create_engine(f'postgresql://{connection_string}', echo=False)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
inspector = inspect(engine)

In [5]:
columns = inspector.get_columns('c19')
for column in columns:
    print(column["name"], column["type"])

dateRep VARCHAR(50)
day INTEGER
month INTEGER
year INTEGER
cases INTEGER
deaths INTEGER
countriesAndTerritories VARCHAR(255)
geoId VARCHAR(255)
countryterritoryCode VARCHAR(255)
popData2018 INTEGER


In [43]:
from flask import jsonify
import json
import numpy as np
import requests

In [7]:
user="gtuser"
password="rpython2020"
host="104.197.208.53"
port="5432"
database="covid"
connection_string = f"{user}:{password}@{host}:{port}/{database}"
engine = create_engine(f'postgresql://{connection_string}')
tables="country"

In [10]:
conn = psycopg2.connect(host=host, port = port, database=database, user=user, password=password)
cur = conn.cursor()
cur.execute(f"""SELECT * FROM {tables}""")
results = cur.fetchall()
conn.close()

In [9]:
v=(str(list(np.ravel(results)))).replace('Decimal(','').replace(')','')

In [46]:
def getLatitudeLongitude(iCountry):
    conn = psycopg2.connect(host=host, port = port, database=database, user=user, password=password)
    cur = conn.cursor()
    query="SELECT latitude,longitude FROM country where name="+"'"+iCountry+"'"
    cur.execute(query)
    results = cur.fetchall()
    conn.close()
    return results

In [167]:
iCountry="Andorra"
LatLong=str(getLatitudeLongitude(iCountry)).replace('Decimal','').replace(")",'').replace("(",'').replace("'","").replace("[","").replace("]","")
LatLong.split(",")[0]
LatLong.split(",")[1]

# print(str(getLatitudeLongitude(iCountry)).split(",")[1])

' 1.601554'

In [251]:
country=pd.read_csv("../static/data/country.csv").drop(columns=['country'])
country.rename(columns = {'name':'country'}, inplace = True) 
country

,latitude,longitude,country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla
...,...,...,...
240,15.552727,48.516388,Yemen
241,-12.827500,45.166244,Mayotte
242,-30.559482,22.937506,South Africa
243,-13.133897,27.849332,Zambia


In [34]:
url="https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
#response = requests.get(url).json()
response = requests.get(url)
response_text=json.loads(response.text)
response_jn = pd.json_normalize(response_text)
c19df = pd.DataFrame(response_jn)
#data_c19=json.dumps(response, indent=4, sort_keys=True)

In [253]:
url="https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
response = requests.get(url).json()
data=response["records"]
dateRep = [item['dateRep'].replace("/","-") for item in data]
cases = [pd.to_numeric(item['cases']) for item in data]
deaths = [pd.to_numeric(item['deaths']) for item in data]
countriesAndTerritories = [item['countriesAndTerritories'] for item in data]
popData2018 = [pd.to_numeric(item['popData2018']) for item in data]

df=  pd.DataFrame(
  {'dateRep': dateRep,
   'cases': cases,
   'deaths': deaths,
   'country': countriesAndTerritories,
   'popData2018': popData2018,
  })

In [254]:
df.head()

,dateRep,cases,deaths,country,popData2018
0,19-04-2020,63,0,Afghanistan,37172386.0
1,18-04-2020,51,1,Afghanistan,37172386.0
2,17-04-2020,10,4,Afghanistan,37172386.0
3,16-04-2020,70,2,Afghanistan,37172386.0
4,15-04-2020,49,2,Afghanistan,37172386.0


In [255]:
# Combine the data into a single dataset
c19_data = pd.merge(df, country, how="left", on=["country", "country"])

In [260]:
c19_data.to_json(orient="records")

'[{"dateRep":"19-04-2020","cases":63,"deaths":0,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"18-04-2020","cases":51,"deaths":1,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"17-04-2020","cases":10,"deaths":4,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"16-04-2020","cases":70,"deaths":2,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"15-04-2020","cases":49,"deaths":2,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"14-04-2020","cases":58,"deaths":3,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"13-04-2020","cases":52,"deaths":0,"country":"Afghanistan","popData2018":37172386.0,"latitude":33.93911,"longitude":67.709953},{"dateRep":"12-04-2020","cases":

In [258]:
#c19_data.json("../static/data/covidfulldata.").reset_index().drop(columns=['index']).to_json(orient="records")
c19_data.to_csv("../static/data/covidfulldata.csv",index=False)

In [235]:
countrygroup=df.groupby("countriesAndTerritories")

In [236]:
countrygroup.head()

,dateRep,cases,deaths,countriesAndTerritories,popData2018
0,19-04-2020,63,0,Afghanistan,37172386.0
1,18-04-2020,51,1,Afghanistan,37172386.0
2,17-04-2020,10,4,Afghanistan,37172386.0
3,16-04-2020,70,2,Afghanistan,37172386.0
4,15-04-2020,49,2,Afghanistan,37172386.0
...,...,...,...,...,...
11738,19-04-2020,1,0,Zimbabwe,14439018.0
11739,18-04-2020,0,0,Zimbabwe,14439018.0
11740,17-04-2020,1,0,Zimbabwe,14439018.0
11741,16-04-2020,6,0,Zimbabwe,14439018.0


In [239]:
TotalCases=countrygroup['cases'].sum()
TotalLosses=countrygroup['deaths'].sum()
Date = countrygroup["dateRep"].first()
Population=countrygroup["popData2018"].first()

In [240]:
#Build Purchase Dataframe with Pre-calculated Values
newdf=pd.DataFrame({"TotalCases":TotalCases,"TotalLosses": TotalLosses,"Date":Date,"Population":Population})
newdf.head(10)

,TotalCases,TotalLosses,Date,Population
countriesAndTerritories,,,,
Afghanistan,908,30,19-04-2020,37172386.0
Albania,548,26,19-04-2020,2866376.0
Algeria,2535,367,19-04-2020,42228429.0
Andorra,704,35,19-04-2020,77006.0
Angola,24,2,19-04-2020,30809762.0
Anguilla,3,0,19-04-2020,NaN
Antigua_and_Barbuda,23,2,19-04-2020,96286.0
Argentina,2828,132,19-04-2020,44494502.0
Armenia,1291,20,19-04-2020,2951776.0


In [247]:
alldf=pd.DataFrame()
for index,row in newdf.iterrows():
    try:

        iCountry=index
        LatLon=pd.DataFrame(getLatitudeLongitude(iCountry))
        #print(LatLon[0])
        tmp=pd.DataFrame({"Country":[index],"Latitude":LatLon[0],"Longitude":LatLon[1],"TotalCases":[row['TotalCases']],"TotalLosses": [row['TotalLosses']],"Date":[row['Date']],"Population":[row['Population']]})

    except KeyError:

        tmp=pd.DataFrame({"Country":[index],"Latitude":"","Longitude":"","TotalCases":[row['TotalCases']],"TotalLosses": [row['TotalLosses']],"Date":[row['Date']],"Population":[row['Population']]})

    #tmp['Latitude']=LatLon[0]
    #tmp['Longitude']=LatLon[1]
    alldf=alldf.append(tmp)


OperationalError: could not connect to server: Network is unreachable
	Is the server running on host "104.197.208.53" and accepting
	TCP/IP connections on port 5432?


In [242]:
n=alldf.loc[alldf.Latitude != ''].reset_index()
n

,index,Country,Latitude,Longitude,TotalCases,TotalLosses,Date,Population
0,0,Afghanistan,33.93911,67.709953,908,30,19-04-2020,37172386.0
1,0,Albania,41.153332,20.168331,548,26,19-04-2020,2866376.0
2,0,Algeria,28.033886,1.659626,2535,367,19-04-2020,42228429.0
3,0,Andorra,42.546245,1.601554,704,35,19-04-2020,77006.0
4,0,Angola,-11.202692,17.873887,24,2,19-04-2020,30809762.0
5,0,Anguilla,18.220554,-63.068615,3,0,19-04-2020,NaN
6,0,Argentina,-38.416097,-63.616672,2828,132,19-04-2020,44494502.0
7,0,Armenia,40.069099,45.038189,1291,20,19-04-2020,2951776.0


In [246]:
data_fullC19=n.drop(columns=['index']).to_csv("test.csv",index=False)
data_fullC19

In [14]:
df=pd.DataFrame()
for r in results:
    tmp_df=pd.DataFrame({"name":[r[0]],"country":[r[1]],"latitude":[r[2]],"longitude":[r[3]]})
    df=df.append(tmp_df)

In [16]:
df=df.reset_index().drop(columns=['index']).to_json(orient="records")

'[{"name":"Andorra","country":"AD","latitude":42.546245,"longitude":1.601554},{"name":"United Arab Emirates","country":"AE","latitude":23.424076,"longitude":53.847818},{"name":"Afghanistan","country":"AF","latitude":33.93911,"longitude":67.709953},{"name":"Antigua and Barbuda","country":"AG","latitude":17.060816,"longitude":-61.796428},{"name":"Anguilla","country":"AI","latitude":18.220554,"longitude":-63.068615},{"name":"Albania","country":"AL","latitude":41.153332,"longitude":20.168331},{"name":"Armenia","country":"AM","latitude":40.069099,"longitude":45.038189},{"name":"Netherlands Antilles","country":"AN","latitude":12.226079,"longitude":-69.060087},{"name":"Angola","country":"AO","latitude":-11.202692,"longitude":17.873887},{"name":"Antarctica","country":"AQ","latitude":-75.250973,"longitude":-0.071389},{"name":"Argentina","country":"AR","latitude":-38.416097,"longitude":-63.616672},{"name":"American Samoa","country":"AS","latitude":-14.270972,"longitude":-170.132217},{"name":"Aus

In [177]:
#df=df[['name','country','latitude','longitude']]
df.

'[{"name":"Andorra","country":"AD","latitude":42.546245,"longitude":1.601554},{"name":"United Arab Emirates","country":"AE","latitude":23.424076,"longitude":53.847818},{"name":"Afghanistan","country":"AF","latitude":33.93911,"longitude":67.709953},{"name":"Antigua and Barbuda","country":"AG","latitude":17.060816,"longitude":-61.796428},{"name":"Anguilla","country":"AI","latitude":18.220554,"longitude":-63.068615},{"name":"Albania","country":"AL","latitude":41.153332,"longitude":20.168331},{"name":"Armenia","country":"AM","latitude":40.069099,"longitude":45.038189},{"name":"Netherlands Antilles","country":"AN","latitude":12.226079,"longitude":-69.060087},{"name":"Angola","country":"AO","latitude":-11.202692,"longitude":17.873887},{"name":"Antarctica","country":"AQ","latitude":-75.250973,"longitude":-0.071389},{"name":"Argentina","country":"AR","latitude":-38.416097,"longitude":-63.616672},{"name":"American Samoa","country":"AS","latitude":-14.270972,"longitude":-170.132217},{"name":"Aus

In [109]:
import pandas as pd
i=0
for r in results:
    if i==0:
        rav=list(np.ravel(r))
        pd.DataFrame(rav)
    i=i+1
    
#dec = decimal.Decimal('10.0')
#str(dec)

In [118]:
df=pd.DataFrame(results)
dfnew=df.rename(columns={"0": "name","1":"country","2":"latitude","3":"longitude"})
dfnew

,0,1,2,3
0,Andorra,AD,42.546245,1.601554
1,United Arab Emirates,AE,23.424076,53.847818
2,Afghanistan,AF,33.93911,67.709953
3,Antigua and Barbuda,AG,17.060816,-61.796428
4,Anguilla,AI,18.220554,-63.068615
...,...,...,...,...
240,Yemen,YE,15.552727,48.516388
241,Mayotte,YT,-12.8275,45.166244
242,South Africa,ZA,-30.559482,22.937506
243,Zambia,ZM,-13.133897,27.849332


In [145]:
pd.DataFrame(results).to_json(orient='columns')

'{"0":{"0":"Andorra","1":"United Arab Emirates","2":"Afghanistan","3":"Antigua and Barbuda","4":"Anguilla","5":"Albania","6":"Armenia","7":"Netherlands Antilles","8":"Angola","9":"Antarctica","10":"Argentina","11":"American Samoa","12":"Austria","13":"Australia","14":"Aruba","15":"Azerbaijan","16":"Bosnia and Herzegovina","17":"Barbados","18":"Bangladesh","19":"Belgium","20":"Burkina Faso","21":"Bulgaria","22":"Bahrain","23":"Burundi","24":"Benin","25":"Bermuda","26":"Brunei","27":"Bolivia","28":"Brazil","29":"Bahamas","30":"Bhutan","31":"Bouvet Island","32":"Botswana","33":"Belarus","34":"Belize","35":"Canada","36":"Cocos [Keeling] Islands","37":"Congo [DRC]","38":"Central African Republic","39":"Congo [Republic]","40":"Switzerland","41":"C\\u00f4te d\'Ivoire","42":"Cook Islands","43":"Chile","44":"Cameroon","45":"China","46":"Colombia","47":"Costa Rica","48":"Cuba","49":"Cape Verde","50":"Christmas Island","51":"Cyprus","52":"Czech Republic","53":"Germany","54":"Djibouti","55":"Denma

In [71]:
w.replace("'[",'"[')

'["Andorra", "AD", "42.546245", "1.601554", "United Arab Emirates", "AE", "23.424076", "53.847818", "Afghanistan", "AF", "33.93911", "67.709953", "Antigua and Barbuda", "AG", "17.060816", "-61.796428", "Anguilla", "AI", "18.220554", "-63.068615", "Albania", "AL", "41.153332", "20.168331", "Armenia", "AM", "40.069099", "45.038189", "Netherlands Antilles", "AN", "12.226079", "-69.060087", "Angola", "AO", "-11.202692", "17.873887", "Antarctica", "AQ", "-75.250973", "-0.071389", "Argentina", "AR", "-38.416097", "-63.616672", "American Samoa", "AS", "-14.270972", "-170.132217", "Austria", "AT", "47.516231", "14.550072", "Australia", "AU", "-25.274398", "133.775136", "Aruba", "AW", "12.52111", "-69.968338", "Azerbaijan", "AZ", "40.143105", "47.576927", "Bosnia and Herzegovina", "BA", "43.915886", "17.679076", "Barbados", "BB", "13.193887", "-59.543198", "Bangladesh", "BD", "23.684994", "90.356331", "Belgium", "BE", "50.503887", "4.469936", "Burkina Faso", "BF", "12.238333", "-1.561593", "Bul

In [ ]:
table="c19"
i=0
columns = inspector.get_columns(table)
for column in columns:
    if i==0:
        queryList= table+"."+column["name"]
    else:
        queryList= queryList+ ","+ table+"."+column["name"]
    i=i+1
    
queryList
    

In [ ]:
session = Session(engine)
results = session.query(queryList).all()
session.close()
len(results)

In [ ]:
LatestDate=np.ravel(session.query(Measurement.date).order_by(Measurement.date.desc()).first())